In [9]:
import os
import pandas as pd
import tensorflow as tf
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Bidirectional, Conv1D, GlobalMaxPooling1D, Concatenate, AdditiveAttention
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import AdditiveAttention
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from utils import eval, eval_training, get_train_test_data
import re
import numpy as np
import spacy
from gensim.models import Word2Vec
from transformers import BertTokenizer, TFBertModel


os.environ['TF_NUM_INTEROP_THREADS'] = '10'
os.environ['TF_NUM_INTRAOP_THREADS'] = '10'

# Configure TensorFlow session for multi-threading
tf.config.threading.set_inter_op_parallelism_threads(10)
tf.config.threading.set_intra_op_parallelism_threads(10)
# Ensure TensorFlow is using the Metal backend
gpu = len (tf.config.list_physical_devices ('GPU'))>0
print ("GPU is", "available" if gpu else "NOT AVAILABLE")

GPU is available


In [10]:
data_label = "data"
predict_label = "label"
balanced = False
labels = ["meniskus_urgent", "cruciate_urgent"]
class_weights = False
sample_weights = False
df = pd.read_csv('../data/balanced_classification_dataset.csv')

df = df.sample(frac=1).reset_index(drop=True)
if labels is not None:
  df = df[df['label'].isin(labels)]

# Split data into training and temporary data (the remaining 40%)
train_df, temp_df = train_test_split(df, test_size=0.4, random_state=42, stratify=df[predict_label])

# Split the temporary data into validation and test sets (each 50% of temporary, thus 20% of total each)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df[predict_label])

# Check the size of each set
print("Training set size:", len(train_df))
print("Validation set size:", len(valid_df))
print("Test set size:", len(test_df))
# Prepare data for training
train_texts = train_df[data_label]
valid_texts = valid_df[data_label]
test_texts = test_df[data_label]

# Initialize the label encoder
all_labels = pd.concat([train_df[predict_label], valid_df[predict_label], test_df[predict_label]])
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)
# Fit label encoder and return encoded labels as integers
train_labels_enc = label_encoder.transform(train_df[predict_label])
valid_labels_enc = label_encoder.transform(valid_df[predict_label])
test_labels_enc = label_encoder.transform(test_df[predict_label])

# Convert labels to categorical one-hot encoding
train_labels = to_categorical(train_labels_enc)
valid_labels = to_categorical(valid_labels_enc)
test_labels = to_categorical(test_labels_enc)

num_classes = len(train_df[predict_label].unique())

Training set size: 14946
Validation set size: 4982
Test set size: 4982


In [4]:
# Define model parameters
embedding_dim = 300  # Dimension of the embedding vectors
lstm_units = 64
epochs = 10
batch_size = 32

# No Embedding Model

In [6]:
optimizer = Adam()
loss = tf.keras.losses.CategoricalCrossentropy()

metrics=[
      tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.F1Score(name='f1_score'),
]

2024-07-22 11:48:56.102126: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2024-07-22 11:48:56.102153: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2024-07-22 11:48:56.102157: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2024-07-22 11:48:56.102170: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-22 11:48:56.102184: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
# Tokenize texts
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_texts)  # Only fit on train data
word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(train_texts)
valid_sequences = tokenizer.texts_to_sequences(valid_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)
max_length = max(len(sequence) for sequence in train_sequences) + 30
print(max_length)
# Padding sequences
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')
valid_padded = pad_sequences(valid_sequences, maxlen=max_length, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

304


In [6]:
# Define the input layer
input_text = Input(shape=(max_length,), dtype='int32', name='text_input')

# Embedding layer
embedding = Embedding(input_dim=len(word_index), output_dim=embedding_dim)(input_text)

# Two LSTM layers
x = Bidirectional(LSTM(units=lstm_units, return_sequences=True))(embedding)
x = Dropout(0.5)(x)
x = Bidirectional(LSTM(units=lstm_units))(embedding)


# Output layer
output = Dense(num_classes, activation='softmax')(x)

# Define the model
model_base = Model(inputs=input_text, outputs=output)

# Compile the model
model_base.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [7]:
history_base = model_base.fit(train_padded, train_labels, epochs=epochs, batch_size=batch_size,
                    validation_data=(valid_padded, valid_labels))

Epoch 1/10


2024-07-21 17:01:30.511902: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


425/425 ━━━━━━━━━━━━━━━━━━━━ 53s 114ms/step - accuracy: 0.4673 - auc: 0.8264 - f1_score: 0.2134 - loss: 1.2916 - val_accuracy: 0.5768 - val_auc: 0.8809 - val_f1_score: 0.3012 - val_loss: 1.0893
Epoch 2/10
425/425 ━━━━━━━━━━━━━━━━━━━━ 47s 111ms/step - accuracy: 0.6093 - auc: 0.8894 - f1_score: 0.3176 - loss: 1.0492 - val_accuracy: 0.5828 - val_auc: 0.8856 - val_f1_score: 0.3058 - val_loss: 1.0687
Epoch 3/10
425/425 ━━━━━━━━━━━━━━━━━━━━ 46s 109ms/step - accuracy: 0.6710 - auc: 0.9115 - f1_score: 0.3510 - loss: 0.9337 - val_accuracy: 0.5919 - val_auc: 0.8881 - val_f1_score: 0.3085 - val_loss: 1.0558
Epoch 4/10
425/425 ━━━━━━━━━━━━━━━━━━━━ 46s 108ms/step - accuracy: 0.7102 - auc: 0.9277 - f1_score: 0.3827 - loss: 0.8423 - val_accuracy: 0.5911 - val_auc: 0.8866 - val_f1_score: 0.3268 - val_loss: 1.0754
Epoch 5/10
425/425 ━━━━━━━━━━━━━━━━━━━━ 46s 108ms/step - accuracy: 0.7514 - auc: 0.9427 - f1_score: 0.4462 - loss: 0.7437 - val_accuracy: 0.5881 - val_auc: 0.8817 - val_f1_score: 0.3276 - val

# Word2Vec Embedding Model

In [7]:
optimizer = Adam()
loss = tf.keras.losses.CategoricalCrossentropy()
embedding_dim = 300
metrics=[
      tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.F1Score(average="macro", name='f1_score'),
]

In [6]:
# Load spaCy model
nlp = spacy.load('de_core_news_sm')

def preprocess_text(text):
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-ZäöüÄÖÜß\s]', '', text)
    doc = nlp(text)
    stop_words = spacy.lang.de.stop_words.STOP_WORDS
    words = [token.text for token in doc if token.text.lower() not in stop_words and token.is_alpha]
    return words

def get_word2vec_embeddings(text, model, vector_size=300):
    tokens = preprocess_text(text)
    embeddings = np.zeros((len(tokens), vector_size))
    for i, token in enumerate(tokens):
        if token in model.wv:
            embeddings[i] = model.wv[token]
        else:
            embeddings[i] = np.zeros(vector_size)
    return embeddings

# Load pre-trained Word2Vec embeddings
w2v = Word2Vec.load("../data/word2vec.model")

In [8]:
w2v_train_embeddings = [get_word2vec_embeddings(text, w2v) for text in train_texts]
w2v_valid_embeddings = [get_word2vec_embeddings(text, w2v) for text in valid_texts]
w2v_test_embeddings = [get_word2vec_embeddings(text, w2v) for text in test_texts]
max_length = max(len(embedding) for embedding in w2v_train_embeddings)
w2v_padded_train_embeddings = pad_sequences(w2v_train_embeddings, maxlen=max_length, padding='post', dtype='float32')
w2v_padded_valid_embeddings = pad_sequences(w2v_valid_embeddings, maxlen=max_length, padding='post', dtype='float32')
w2v_padded_test_embeddings = pad_sequences(w2v_test_embeddings, maxlen=max_length, padding='post', dtype='float32')


# Define the input layer
input_text = Input(shape=(max_length, embedding_dim), dtype='float32', name='text_input')

# Two LSTM layers
x = Bidirectional(LSTM(units=lstm_units, return_sequences=True))(input_text)
x = Dropout(0.5)(x)
x = Bidirectional(LSTM(units=lstm_units))(embedding)


# Output layer
output = Dense(num_classes, activation='softmax')(x)

# Define the model
model_word2vec = Model(inputs=input_text, outputs=output)

# Compile the model
model_word2vec.compile(optimizer=optimizer, loss=loss, metrics=metrics)

NameError: name 'embedding' is not defined

In [10]:
history_word2vec = model_word2vec.fit(w2v_padded_train_embeddings, train_labels, 
                                      epochs=10, batch_size=32, 
                                      validation_data=(w2v_padded_valid_embeddings, valid_labels))

Epoch 1/10


2024-07-21 21:03:31.992280: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


849/849 ━━━━━━━━━━━━━━━━━━━━ 253s 290ms/step - accuracy: 0.3981 - auc: 0.7935 - f1_score: 0.1251 - loss: 1.3708 - val_accuracy: 0.4085 - val_auc: 0.8129 - val_f1_score: 0.1545 - val_loss: 1.3150
Epoch 2/10
849/849 ━━━━━━━━━━━━━━━━━━━━ 242s 285ms/step - accuracy: 0.4007 - auc: 0.8069 - f1_score: 0.1287 - loss: 1.3239 - val_accuracy: 0.4075 - val_auc: 0.8172 - val_f1_score: 0.0966 - val_loss: 1.2927
Epoch 3/10
849/849 ━━━━━━━━━━━━━━━━━━━━ 243s 286ms/step - accuracy: 0.4078 - auc: 0.8097 - f1_score: 0.1327 - loss: 1.3138 - val_accuracy: 0.3870 - val_auc: 0.8158 - val_f1_score: 0.1510 - val_loss: 1.2949
Epoch 4/10
849/849 ━━━━━━━━━━━━━━━━━━━━ 242s 285ms/step - accuracy: 0.4166 - auc: 0.8161 - f1_score: 0.1651 - loss: 1.2945 - val_accuracy: 0.4555 - val_auc: 0.8387 - val_f1_score: 0.1920 - val_loss: 1.2226
Epoch 5/10
849/849 ━━━━━━━━━━━━━━━━━━━━ 242s 285ms/step - accuracy: 0.4216 - auc: 0.8191 - f1_score: 0.1630 - loss: 1.2867 - val_accuracy: 0.4578 - val_auc: 0.8414 - val_f1_score: 0.1920 

# Bert Embedding Model

In [5]:
optimizer = Adam()
loss = tf.keras.losses.CategoricalCrossentropy()

metrics=[
      tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.F1Score(name='f1_score'),
]

2024-07-22 19:50:09.361011: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2024-07-22 19:50:09.361049: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2024-07-22 19:50:09.361057: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2024-07-22 19:50:09.361084: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-22 19:50:09.361099: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
from transformers import BertTokenizer, BertModel
import torch
import concurrent.futures
import pickle
import numpy as np

# Function to generate word embeddings for a given batch of texts
def generate_bert_embeddings_batch(texts, tokenizer, model, max_length):
    inputs = tokenizer(texts, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state
    return embeddings.numpy()

model_name = "GerMedBERT/medbert-512"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Function to generate embeddings concurrently in batches
def generate_embeddings_concurrently(texts, tokenizer, model, max_length, batch_size=32):
    embeddings = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            futures.append(executor.submit(generate_bert_embeddings_batch, batch_texts, tokenizer, model, max_length))
        
        for future in concurrent.futures.as_completed(futures):
            try:
                batch_embeddings = future.result()
                embeddings.extend(batch_embeddings)
            except Exception as exc:
                print(f'Generated an exception: {exc}')
    return np.array(embeddings)

# Ensure texts are in list format
train_texts = train_texts.tolist()
valid_texts = valid_texts.tolist()
test_texts = test_texts.tolist()

# Define the max length for padding
max_length = 400  # or set to a specific value based on your data
print("Starting")

# Generate embeddings concurrently in batches
bert_train_embeddings = generate_embeddings_concurrently(train_texts, tokenizer, model, max_length)
bert_padded_train_embeddings = np.pad(bert_train_embeddings, ((0,0),(0,max_length-bert_train_embeddings.shape[1]),(0,0)), 'constant')
# Save padded embeddings to files
with open('bert_train.pkl', 'wb') as file:
    pickle.dump(bert_padded_train_embeddings, file)
print("Training done")

bert_valid_embeddings = generate_embeddings_concurrently(valid_texts, tokenizer, model, max_length)
bert_padded_valid_embeddings = np.pad(bert_valid_embeddings, ((0,0),(0,max_length-bert_valid_embeddings.shape[1]),(0,0)), 'constant')
with open('bert_valid.pkl', 'wb') as file:
    pickle.dump(bert_padded_valid_embeddings, file)
print("Valid done")

bert_test_embeddings = generate_embeddings_concurrently(test_texts, tokenizer, model, max_length)
bert_padded_test_embeddings = np.pad(bert_test_embeddings, ((0,0),(0,max_length-bert_test_embeddings.shape[1]),(0,0)), 'constant')
with open('bert_test.pkl', 'wb') as file:
    pickle.dump(bert_padded_test_embeddings, file)
print("Test done")


Some weights of BertModel were not initialized from the model checkpoint at GerMedBERT/medbert-512 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting
Training done
Valid done
Test done


In [7]:
import pickle
with open("bert_train.pkl", "rb") as input_file:
    bert_padded_train_embeddings = pickle.load(input_file)
with open("bert_valid.pkl", "rb") as input_file:
    bert_padded_valid_embeddings = pickle.load(input_file)
with open("bert_test.pkl", "rb") as input_file:
    bert_padded_test_embeddings = pickle.load(input_file)

In [15]:
embedding_dim = bert_padded_train_embeddings.shape[-1]
max_length = 400
# Define the input layers
input_text = Input(shape=(max_length, embedding_dim), dtype='float32', name='text_input')

# Two LSTM layers
x = Bidirectional(LSTM(units=lstm_units, return_sequences=True))(input_text)
x = Dropout(0.5)(x)
x = Bidirectional(LSTM(units=lstm_units))(x)

# Output layer
output = Dense(num_classes, activation='softmax')(x)

# Define the model
model_bert = Model(inputs=input_text, outputs=output)

# Define the model

# Compile the model
model_bert.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [16]:
history_bert = model_bert.fit(bert_padded_train_embeddings, train_labels, 
                                      epochs=10, batch_size=32, 
                                      validation_data=(bert_padded_valid_embeddings, valid_labels))

Epoch 1/10


2024-07-22 19:56:56.973191: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


468/468 ━━━━━━━━━━━━━━━━━━━━ 110s 230ms/step - accuracy: 0.7369 - auc: 0.7446 - f1_score: 0.4403 - loss: 0.5835 - val_accuracy: 0.7403 - val_auc: 0.7348 - val_f1_score: 0.4254 - val_loss: 0.5737
Epoch 2/10
468/468 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - accuracy: 0.7402 - auc: 0.7408 - f1_score: 0.4256 - loss: 0.5757

KeyboardInterrupt: 

# Evaluation

In [ ]:
eval_training(history_base, metrics)
eval(model_base, test_padded, test_labels, label_encoder)

In [ ]:
eval_training(history_word2vec, metrics)
eval(model_word2vec, w2v_padded_test_embeddings, test_labels, label_encoder)